In [ ]:
# Import necessary libraries
import os
import numpy as np
import cv2
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [ ]:
# Define paths
dataset_path = '/kaggle/input/1000-videos-split/1000_videos'#change the data set path
train_path = os.path.join(dataset_path, 'train')
test_path = os.path.join(dataset_path, 'test')
validation_path = os.path.join(dataset_path, 'validation')

In [ ]:
# Define image size and channels
Image_Size = 180  # Update this as necessary
Channels = 3
Epochs = 100 # Update this as necessary

In [ ]:
# Create a function to load and preprocess images
def load_images(image_path, label):
    images = []
    labels = []
    for img in os.listdir(image_path):
        img_path = os.path.join(image_path, img)
        img_data = cv2.imread(img_path)
        img_data = cv2.resize(img_data, (Image_Size, Image_Size))
        images.append(img_data)
        labels.append(label)
    return np.array(images), np.array(labels)


In [ ]:
# Load training data
train_real_images, train_real_labels = load_images('/kaggle/input/1000-videos-split/1000_videos/train/real', 0) #change the path
train_fake_images, train_fake_labels = load_images('/kaggle/input/1000-videos-split/1000_videos/train/fake', 1) #change the path

# Load validation data 
validation_real_images, validation_real_labels = load_images('/kaggle/input/1000-videos-split/1000_videos/validation/real', 0) #change the path
validation_fake_images, validation_fake_labels = load_images('/kaggle/input/1000-videos-split/1000_videos/validation/fake', 1) #change the path

# Load test data  # Load the test data here
test_real_images, test_real_labels = load_images('/kaggle/input/1000-videos-split/1000_videos/test/real', 0)  #change the path
test_fake_images, test_fake_labels = load_images('/kaggle/input/1000-videos-split/1000_videos/test/fake', 1) #change the path

# Combine real and fake datasets
x_train = np.concatenate((train_real_images, train_fake_images), axis=0)
y_train = np.concatenate((train_real_labels, train_fake_labels), axis=0)

x_validation = np.concatenate((validation_real_images, validation_fake_images), axis=0)
y_validation = np.concatenate((validation_real_labels, validation_fake_labels), axis=0)

x_test = np.concatenate((test_real_images, test_fake_images), axis=0) 
y_test = np.concatenate((test_real_labels, test_fake_labels), axis=0)

In [ ]:
# Normalize the images
x_train = x_train / 255.0
x_validation = x_validation / 255.0
x_test = x_test / 255.0

print(f'Training data shape: {x_train.shape}, Labels shape: {y_train.shape}')
print(f'Validation data shape: {x_validation.shape}, Labels shape: {y_validation.shape}')
print(f'Test data shape: {x_test.shape}, Labels shape: {y_test.shape}')


In [ ]:
# Build the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(Image_Size, Image_Size, Channels)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=Epochs, validation_data=(x_validation, y_validation))


In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

In [ ]:
# Define the prediction function for a video
def pred_video(model, video_path):
    frames = []
    img_data = cv2.imread(video_path)
    img_data = cv2.resize(img_data, (Image_Size, Image_Size))
    img_data = img_data / 255.0
    frames.append(img_data)
    frames = np.array(frames)
    predictions = model.predict(frames)
    final_prediction = np.mean(predictions)
    return final_prediction, predictions
# 


In [ ]:
# Define the prediction function for individual images
def pred(model, image):
    image = tf.image.resize(image, [Image_Size, Image_Size])  # Resize to match the input shape
    image = np.expand_dims(image, axis=0)  # Expand dims to match input shape
    prediction = model.predict(image)
    predicted_class = 'fake' if prediction[0] > 0.5 else 'real'
    confidence = prediction[0][0] * 100 if predicted_class == 'fake' else (1 - prediction[0][0]) * 100
    return predicted_class, confidence
# 

In [ ]:
# Save the model
import pickle
with open('model_f_real_pickle_final', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# Load the saved model using pickle
with open('model_f_real_pickle_final', 'rb') as f:  # Update this path
    model_saved = pickle.load(f)


In [ ]:
# Assuming you have class names
class_names = ['fake', 'real']

In [ ]:


# Function to load and preprocess an image from a given path
def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.resize(image, [Image_Size, Image_Size])
    image = image / 255.0  # Normalize to [0, 1]
    return image


In [ ]:

# Predict using the saved model
plt.figure(figsize=(15, 15))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)  # Create a dataset for testing
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        img = images[i].numpy().astype("uint8")
        img_display = img * 255.0  # Scale back to [0, 255] range
        plt.imshow(img_display.astype("uint8"))
        predicted_class, confidence = pred(model_saved, images[i].numpy())
        actual_class = class_names[labels[i]]
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence:.2f}%")
        plt.axis("off")
plt.show()


In [ ]:

# Add an image of your choice
image_path = '/kaggle/input/1000-videos-split/1000_videos/test/real/069_20.png' # Update this path
image = load_and_preprocess_image(image_path)
# Plot and predict for the new image
plt.figure(figsize=(6, 6))
plt.imshow(image.numpy())
predicted_class, confidence = pred(model_saved, image.numpy())
plt.title(f"Predicted: {predicted_class}\nConfidence: {confidence:.2f}%")
plt.axis("off")
plt.show()

